# Pretrained Model
### The reason that it is introduced
* [ILSCVRC](http://image-net.org/)에서 1000개의 클래스를 classification하는 문제를 solving한 모델이 있다.
* 이모델은 1000개를 분류할 수 있는 능력이 있지만 수백만장의 이미지 데이터를 다시 트레이닝하는 것은 매우 무리다.
* 그래서 연구소에서는 이모델을 open source로 제공하는 방향으로 연구를 하고 이를 우리는 사용해 좋은 **visual features **를 이용할 수 있게 되었다.
* 사용법은 매우 간단하다.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [ ]:
transform_train = transforms.Compose([
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                      download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                         shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                      download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                        shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

## Check dataloader, input datas

In [ ]:
# implement this
dataiter = iter(testloader)
data_dict = dataiter.next()
print(data_dict[0].shape, data_dict[1][0])
ex_input = data_dict[0][0]
print(ex_input.shape)
npimg = ex_input.numpy()/2.0 + 0.5
plt.imshow(np.transpose(npimg, (1,2,0)))
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                        shuffle=True, num_workers=2)

## Load and Define pretrained model and fine-tuning

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from torchsummary import summary

In [ ]:
class ResNet18(nn.Module):
    def __init__ (self, use_pretrained):
        super(ResNet18, self).__init__()
        self.resnet = models.resnet18(pretrained=use_pretrained)
        
#         for param in self.resnet.parameters():   # to retrain only last fc layer
#             param.requires_grad = False
            
        num_ftrs = self.resnet.fc.in_features
        
        self.resnet.fc = nn.Linear(num_ftrs, 10) # resnet의 마지막 fc layer만 새롭게 초기화
        
    def forward(self, x):
        x = self.resnet(x)
        return x

## Load Model

In [ ]:
import torch.optim as optim


net_pretrained = ResNet18(use_pretrained = True)
print(net_pretrained)
summary(net_pretrained, batch_size=-1, input_size=(3, 32, 32), device='cpu')
net_pretrained = net_pretrained.to(device)

## Training pretrained model

In [ ]:
training_epochs = 2
learning_rate = 0.01

criterion = nn.CrossEntropyLoss() 
optimizer = optim.SGD(net_pretrained.parameters(), lr=learning_rate, momentum = 0.9)

net_pretrained.train()

for epoch in range(training_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = net_pretrained(inputs) 
        loss =  criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        # print statistics
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' %
                 (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
print('Finished Training')

## Test pretrained model

In [ ]:
class_correct = np.zeros(10)
class_total = np.zeros(10)

net_pretrained.eval()    # dropout, batchnorm off
with torch.no_grad():    # gradient flow off Memory leakage 방지
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = net_pretrained(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels)
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

In [ ]:
# Check outputs
print(outputs)
print(predicted)
print(outputs.shape)

In [ ]:
for i in range(10):
    print('Accuracy of %5s : %2f %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))
print('Mean Accuracy : ', 100*np.sum(class_correct)/np.sum(class_total))

## Training unpretrained model

In [ ]:
net_unpretrained = ResNet18(use_pretrained = False).to(device)
optimizer_unpretrained = optim.SGD(net_unpretrained.parameters(), lr=learning_rate, momentum = 0.9)
net_unpretrained.train()

for epoch in range(training_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer_unpretrained.zero_grad()
        
        # forward + backward + optimize
        outputs = net_unpretrained(inputs) 
        loss =  criterion(outputs, labels)
        loss.backward()
        optimizer_unpretrained.step()
        
        running_loss += loss.item()

        # print statistics
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' %
                 (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
print('Finished Training')

## Test unpretrained model

In [ ]:
class_correct = np.zeros(10)
class_total = np.zeros(10)

net_unpretrained.eval()
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = net_unpretrained(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2f %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))
print('Mean Accuracy : ', 100*np.sum(class_correct)/np.sum(class_total))

## Visualization

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def imshow(img):
    img = img / 2 + 0.5  # Unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
        
# Get a batch of test data
dataiter = iter(testloader)

In [ ]:
inputs, labels = dataiter.next()
outputs_pretrained = net_pretrained(inputs.to(device))
_, predicted_pretrain = torch.max(outputs_pretrained, 1)

outputs_unpretrained = net_unpretrained(inputs.to(device))
_, predicted_unpretrain = torch.max(outputs_unpretrained, 1)

imshow(torchvision.utils.make_grid(inputs))

print('gt label:', ' '.join('%5s' % classes[labels[j]] for j in range(4)))
print('pretrained model prediction:', ' '.join('%5s' % classes[predicted_pretrain[j]] for j in range(4)))
print('unpretrained model prediction:', ' '.join('%5s' % classes[predicted_unpretrain[j]] for j in range(4)))

## Compare with ResNet 152

In [ ]:
class ResNet152(nn.Module):
    def __init__ (self, use_pretrained):
        super(ResNet152, self).__init__()
        self.resnet = models.resnet152(pretrained=use_pretrained)

        num_ftrs = self.resnet.fc.in_features
        
        self.resnet.fc = nn.Linear(num_ftrs, 10)
        
    def forward(self, x):
        x = self.resnet(x)
        return x

In [ ]:
model = ResNet152(use_pretrained=True)

summary(model, batch_size=-1, input_size=(3, 32, 32), device='cpu')
model.to(device)

In [ ]:
training_epochs = 2
learning_rate = 0.01

criterion = nn.CrossEntropyLoss() 
optimizer = optim.SGD(net_pretrained.parameters(), lr=learning_rate, momentum = 0.9)

model.train()

for epoch in range(training_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # implement
        # inputs, labels
        # forward + backward + optimize

        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        # forward + backward + optimize
        optimizer.zero_grad()
        outputs = model(inputs) 
        loss =  criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        # print statistics
        if i % 500 == 499:
            print('[%d, %5d] loss: %.3f' %
                 (epoch + 1, i + 1, running_loss / 500))
            running_loss = 0.0
print('Finished Training')

In [ ]:
class_correct = np.zeros(10)
class_total = np.zeros(10)

model.eval()
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2f %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))
print('Mean Accuracy : ', 100*np.sum(class_correct)/np.sum(class_total))